<a href="https://colab.research.google.com/github/AhmadAghaebrahimian/Optimization/blob/main/Newton/NM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the last practice, we used Newton's method to solve an optimization problem for a function with only one variable. Now, let's consider an optimization problem for functions with a multidimensional variable.

In this exercise, we use Newton's method of Logistic Regression which is a very useful algorithm in Machine Learning. Logistic Regression is a method for classification problems to output discrete values. For example, given an input vector of a tumor with its various measurements, the logistic function classifies it as malignant or benign. Logistic regression has an optimization procedure at its core for which in this exercise, we use the Newton method.

Let's begin with importing dependencies

In [1]:
import numpy as np
import pandas as pd

Now it is time to lead the data. We use the same dataset that we used for Gradient Descent. As you recall, the Tumor dataset consists of 30 different measurements (features) of 569 moles (instances). Each instance is assigned to one label, either Malignant or Benign as the type of the tumor.

Let's load the dataset.

In [ ]:
link = 'https://drive.google.com/uc?export=download&id=1m1s6Q7xQfdWf642OqMkzu2nmvoJ0xK_5'
data = pd.read_csv(link)
# data = data.sample(frac=1).reset_index(drop=True)
d = {'M': 1, 'B': 0} # we map labels from string to integer
data['diagnosis'] = data['diagnosis'].map(d)
print(data.head())

As before, let's separate the Tumor data into standard datasets; 80% for training, 10% for development, and 10% for testing.

In [ ]:
test_idx = int(data.shape[0]*0.9)
val_idx = int(data.shape[0]*0.8)

test_data = data[test_idx:]
val_data = data[val_idx:test_idx]
data = data[:val_idx]

train_Y, train_X = data['diagnosis'], data.drop('diagnosis', axis=1)
val_Y, val_X = val_data['diagnosis'], val_data.drop('diagnosis', axis=1)
test_Y, test_X = test_data['diagnosis'], test_data.drop('diagnosis', axis=1)

print('Training data shape: ', train_X.shape)
print('Validation data shape: ', val_X.shape)
print('Testing data shape: ', test_X.shape)

Also, we need the Sigmoid function.

$s(x) = \frac{1}{1+e^{-f(x)}}$

Let's implement it.

In [4]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

As before, we also need to implement a function to compute accuracy:

In [5]:
def get_accuracy(X, y, theta):
    y_hat = np.array(sigmoid(X.dot(theta)))

    ## Converting y_hat probabilities to prediction, >.5 = 1(M), <.5 = 0(B)
    predictions = np.greater(y_hat, 0.5 * np.ones((y_hat.shape[1], 1)))
    accuracy = np.count_nonzero(np.equal(predictions, y)) / predictions.shape[0] * 100
    return accuracy

Now it is time to implement Newton's method. The generalization of Newton’s method to multidimensional data (also called the Newton-Raphson method) is given by the following update rule using the Hessian matrix:

$\theta_{new} := \theta_{old} - H^{-1} \nabla_\theta \ell(\theta)$

where the Hessian matrix $H(.)$ is represented as:

$H_{ij} = \frac{\partial^2 \ell(\theta)}{\partial\theta_i\partial\theta_j}$

For Logistic Regression, Hessian Matrix is calculated by:

$H = - X^{\top} \omega X$

and the gradient by:

$\nabla_\theta \ell(\theta) = X^{\top} (\hat{y} - y)$

where:

$\omega := diag (\hat{y}(1-\hat{y}))$

Let's implement these steps for one step Newton method:



In [6]:
def newton_step(theta_old, y, X):
    y_hat = np.array(sigmoid(X.dot(theta_old[:, 0])), ndmin=2).T  # probability matrix - N x 1
    omega = np.diag((y_hat * (1 - y_hat))[:, 0])  # N by N diagonal matrix
    hessian = X.T.dot(omega).dot(X)  # 30 by 30 matrix
    gradian = X.T.dot(y - y_hat)  # 30 by 1 matrix

    # Deal with non-invertible hessian
    step = np.dot(np.linalg.inv(hessian + 0.1*np.eye(theta_old.shape[0])), gradian)

    theta_new = theta_old + step
    return theta_new

Let's define a function to check if the model is converged or not. It is done by defining a threshold and checking if it supersedes the difference between $\theta_{old}$ and $\theta_{new}$.

In [7]:
def get_converged(theta_old, theta_new, threshold, iters):
    theta_delta = np.abs(theta_old - theta_new)
    return not (np.any(theta_delta > threshold) and iters < max_iters)

Hyperparameters

In [8]:
max_iters = 10
threshold = 0.01 # convergence tolerance

Now, everything is in place. Let's run everything together.

In [9]:
theta_old, theta_new = np.ones((30, 1)), np.zeros((30, 1))
iter_count = 0
converged = False

while not converged:
    print('Validation Accuracy {}% at Iteration {}'.format(get_accuracy(val_X, val_Y.to_frame(), theta_old), iter_count))
    theta_old = theta_new
    theta_new = newton_step(theta_new, train_Y.to_frame(), train_X)
    iter_count += 1
    converged = get_converged(theta_old, theta_new, threshold, iter_count)

print('Test Accuracy {}% at Iteration {}'.format(get_accuracy(val_X, val_Y.to_frame(), theta_new), iter_count))

Validation Accuracy 21.052631578947366% at Iteration 0
Validation Accuracy 78.94736842105263% at Iteration 1
Validation Accuracy 98.24561403508771% at Iteration 2
Validation Accuracy 100.0% at Iteration 3
Validation Accuracy 96.49122807017544% at Iteration 4
Validation Accuracy 94.73684210526315% at Iteration 5
Validation Accuracy 92.98245614035088% at Iteration 6
Validation Accuracy 92.98245614035088% at Iteration 7
Validation Accuracy 92.98245614035088% at Iteration 8
Validation Accuracy 92.98245614035088% at Iteration 9
Test Accuracy 92.98245614035088% at Iteration 10


Optional Exercise 1: One way to check if the optimization works is to check if the gradient is constantly decreasing or not. Plot the gradient curve to investigate this.

Optional Exercise 2: Compare the results on the Tumor dataset using the Gradient Descent and Newton method regarding performance and throughput.